<a href="https://colab.research.google.com/github/steven4354/jsonformer-test/blob/main/jsonformer_faster_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch-gpu]
!pip install accelerate
!pip install google-colab

# download the latest jsonformer from github
!pip install --upgrade git+https://github.com/1rgs/jsonformer.git#egg=jsonformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 20.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/1rgs/jsonformer.git to /tmp/pip-install-l9y698j5/jsonformer_af0a2c97047f4d80b7ed6f086c23b6e0
  Running command git clone --filter=blob:none --quiet https://github.com/1rgs/json

In [2]:
from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import time
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Set the folder path in Google Drive for model files
google_drive_model_path = "/content/drive/My Drive/Colab Notebooks/dolly-v2-3b"

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Download and save the model and tokenizer to Google Drive if they don't exist
# Use dolly 3b for faster speed
model_name = "databricks/dolly-v2-3b"
if not os.path.exists(google_drive_model_path):
    os.makedirs(google_drive_model_path)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model.save_pretrained(google_drive_model_path)
    tokenizer.save_pretrained(google_drive_model_path)
else:
    model = AutoModelForCausalLM.from_pretrained(google_drive_model_path).to(device)
    tokenizer = AutoTokenizer.from_pretrained(google_drive_model_path)

# Use the loaded model and tokenizer with Jsonformer
from jsonformer.format import highlight_values
from jsonformer.main import Jsonformer

weather_schema = {
    "type": "object",
    "properties": {
        "temperature": {"type": "number"},
    },
}

# time
start_time = time.time()

builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=weather_schema,
    prompt="today's temperature in celsius is",
)

print("Generating...")
output = builder()

# time
end_time = time.time()
elapsed_time = end_time - start_time

highlight_values(output)
print(f"Execution time: {elapsed_time:.4f} seconds")


Mounted at /content/drive


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating...
{
  temperature: 15.0
}
Execution time: 8.6024 seconds


In [3]:
# Do one for car

car = {
  "type": "object",
  "properties": {
    "car": {
      "type": "object",
      "properties": {
        "make": {"type": "string"},
        "model": {"type": "string"},
        "year": {"type": "number"},
        "colors": {
          "type": "array",
          "items": {"type": "string"}
        },
        "features": {
          "type": "object",
          "properties": {
            "audio": {
              "type": "object",
              "properties": {
                "brand": {"type": "string"},
                "speakers": {"type": "number"},
                "hasBluetooth": {"type": "boolean"}
              }
            },
            "safety": {
              "type": "object",
              "properties": {
                "airbags": {"type": "number"},
                "parkingSensors": {"type": "boolean"},
                "laneAssist": {"type": "boolean"}
              }
            },
            "performance": {
              "type": "object",
              "properties": {
                "engine": {"type": "string"},
                "horsepower": {"type": "number"},
                "topSpeed": {"type": "number"}
              }
            }
          }
        }
      }
    },
    "owner": {
      "type": "object",
      "properties": {
        "firstName": {"type": "string"},
        "lastName": {"type": "string"},
        "age": {"type": "number"},
      }
    }
  }
}

# time
start_time = time.time()

builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=car,
    prompt="Generate an example car",
)

print("Generating...")
output = builder()

# time
end_time = time.time()
elapsed_time = end_time - start_time

highlight_values(output)
print(f"Execution time: {elapsed_time:.4f} seconds")


Generating...
{
  car: {
    make: "audi",
    model: "model A8",
    year: 2016.0,
    colors: [
      "blue"
    ],
    features: {
      audio: {
        brand: "sony",
        speakers: 2.0,
        hasBluetooth: True
      },
      safety: {
        airbags: 2.0,
        parkingSensors: True,
        laneAssist: True
      },
      performance: {
        engine: "4.0",
        horsepower: 220.0,
        topSpeed: 220.0
      }
    }
  },
  owner: {
    firstName: "John",
    lastName: "Doe",
    age: 40.0
  }
}
Execution time: 41.4003 seconds
